Author: Daniel Low

Based on tutorials:
- https://github.com/huggingface/notebooks/blob/e1983033bf88432e1e371996e1deec2f6ef1c52a/examples/text_classification.ipynb
- https://huggingface.co/blog/ray-tune
- https://huggingface.co/docs/transformers/training

In [1]:
# python and package Versions used

!python --version # I ran on python==3.10.12

Python 3.10.12


In [2]:
!pip install -q torch==2.0.1 datasets==2.14.3 transformers==4.28.1 accelerate==0.15.0 optuna==3.2.0 evaluate
#imbalanced-learn==0.11.0



In [3]:
import torch

# Check if CUDA is available
if torch.cuda.is_available():
    # Print number of GPUs available
    print("Number of GPUs available:", torch.cuda.device_count())

    for i in range(torch.cuda.device_count()):
        print(f"GPU {i}:")
        print(f"\tName: {torch.cuda.get_device_name(i)}")
        print(f"\tCompute Capability: {torch.cuda.get_device_capability(i)}")
        print(f"\tTotal Memory: {torch.cuda.get_device_properties(i).total_memory / 1e9} GB")
        print(torch.cuda.get_device_properties(i))
        # Additional details can be accessed via `torch.cuda.get_device_properties(i)`
else:
    print("CUDA is not available. Please check your installation and if your hardware supports CUDA.")


CUDA is not available. Please check your installation and if your hardware supports CUDA.


In [4]:
'''
Authors: Daniel M. Low
License: See license in github repository
'''

import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime

pd.set_option("display.max_columns", None)
# pd.options.display.width = 0


on_colab = True



if on_colab:
  from google.colab import drive
  project_name = 'concept_tracker'
  drive.mount('/content/drive')
  input_dir = f'/content/drive/MyDrive/datum/{project_name}/data/ctl/'
  output_dir = f'/content/drive/MyDrive/datum/{project_name}/data/output/lexicon_paper/'
else:
  input_dir = './data/ctl/'
  output_dir = './data/output/lexicon_paper/'

os.makedirs(output_dir, exist_ok=True)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML
from datasets.dataset_dict import DatasetDict
from datasets import Dataset
from datasets import load_dataset, load_metric
from datasets import list_metrics
from imblearn.over_sampling import RandomOverSampler


# Fine-tuning a model on a text classification task

In [6]:
toy = False
regression = True # False for classification
task = "suicide_risk"
text_col = 'text'
y_col = 'y'
model_checkpoint = "roberta-base" # "distilbert-base-uncased"
num_labels = 1 # Use 1 for regression
metrics_to_report = ['Model',  'RMSE','RMSE per value','MAE','MAE per value', 'rho', 'gridsearch', 'Best parameters']
metric_name = "rmse"
validation_key = 'validation'
imbalanced = False # if True, will use custom function. False = leave imabalance or balance through oversampling/undersampling/etc

### training or hyperparameter search arguments

In [7]:

# if finetuning without hypeparameter search
batch_size = 16 #not higher due to GPU memory usage limit issues
if toy:
    epochs = 1
else:
    epochs = 3


# hyperparameter search args
do_hyperparameter_search = True
hyperparameter_shards = 10 # 10 means you use 10th of the data from hyperparameter search
hyperparameter_search_n_trials = 10
hyperparameter_search_direction = 'minimize' # maximize for classification metric, minimize for loss

In [8]:
# if searching with hyperparameter search, options are default of optuna or specified here:


# default optuna decided by transformers: https://github.com/huggingface/transformers/blob/dcbfd93d7aeb14f8ff08a48866d2a68950d4c69a/src/transformers/trainer_utils.py#L248

# https://optuna.readthedocs.io/en/stable/tutorial/10_key_features/002_configurations.html#sphx-glr-tutorial-10-key-features-002-configurations-py

def optuna_hp_space(trial):
    return {
        "learning_rate": trial.suggest_float("learning_rate", 1e-6, 1e-4, log=True),
        "num_train_epochs": trial.suggest_int("num_train_epochs", 1,4, step=1),
        "weight_decay": trial.suggest_float("weight_decay", 1e-10, 1e-3, log=True)
    }


## Loading the dataset

In [9]:
balanced_validation_set = True

train = pd.read_csv(input_dir+'train_10perc_text_balanced.csv', index_col=0)
val = pd.read_csv(input_dir+'val_3perc_text_balanced.csv', index_col=0)
test = pd.read_csv(input_dir+'test_3perc_text_balanced.csv', index_col=0)
print(train.shape, val.shape, test.shape)

if balanced_validation_set:
  from sklearn.model_selection import train_test_split
  train, val = train_test_split(train, test_size=0.2, random_state=42)
  print(train.shape, val.shape, test.shape)

(4129, 3) (1078, 3) (1081, 3)
(3303, 3) (826, 3) (1081, 3)


In [10]:


train = train.dropna()
val = val.dropna()
test = test.dropna()






print(train.shape, test.shape)

if toy:
  train = train.sample(frac = 0.1)
  val = val.sample(frac = 0.1)
  test = test.sample(frac = 0.1)


X_train = train[text_col].values
y_train = train[y_col].values

X_val = val[text_col].values
y_val = val[y_col].values

X_test = test[text_col].values
y_test = test[y_col].values

# Oversample to match method of model with metadata approach
# ros = RandomOverSampler(random_state=0)
# X_train, y_train = ros.fit_resample(X_train.reshape(-1, 1), y_train)
# X_train = X_train.flatten()

print(X_train.shape, y_train.shape)

d = {'train':Dataset.from_dict({'label':y_train,'text':X_train}),
     'validation':Dataset.from_dict({'label':y_val,'text':X_val}),
     'test':Dataset.from_dict({'label':y_test,'text':X_test})
     }




dataset = DatasetDict(d)


(3280, 3) (1071, 3)
(3280,) (3280,)


In [11]:
dataset['train']['label'][:10]

[4.0, 4.0, 2.0, 4.0, 4.0, 4.0, 3.0, 4.0, 3.0, 4.0]

In [12]:
from evaluate import load


metrics_list = list_metrics()
print(len(metrics_list))
print(metrics_list)
if regression:
  metric = load('mse')
else:
  metric = load_metric("f1")



<ipython-input-12-f75b8a59469e>:4: FutureWarning: list_metrics is deprecated and will be removed in the next major version of datasets. Use 'evaluate.list_evaluation_modules' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metrics_list = list_metrics()


215
['accuracy', 'bertscore', 'bleu', 'bleurt', 'brier_score', 'cer', 'character', 'charcut_mt', 'chrf', 'code_eval', 'comet', 'competition_math', 'confusion_matrix', 'coval', 'cuad', 'exact_match', 'f1', 'frugalscore', 'glue', 'google_bleu', 'indic_glue', 'mae', 'mahalanobis', 'mape', 'mase', 'matthews_correlation', 'mauve', 'mean_iou', 'meteor', 'mse', 'nist_mt', 'pearsonr', 'perplexity', 'poseval', 'precision', 'r_squared', 'recall', 'rl_reliability', 'roc_auc', 'rouge', 'sacrebleu', 'sari', 'seqeval', 'smape', 'spearmanr', 'squad', 'squad_v2', 'super_glue', 'ter', 'trec_eval', 'wer', 'wiki_split', 'xnli', 'xtreme_s', 'AlhitawiMohammed22/CER_Hu-Evaluation-Metrics', 'BucketHeadP65/confusion_matrix', 'BucketHeadP65/roc_curve', 'DaliaCaRo/accents_unplugged_eval', 'DarrenChensformer/eval_keyphrase', 'DarrenChensformer/relation_extraction', 'DoctorSlimm/bangalore_score', 'DoctorSlimm/kaushiks_criteria', 'Drunper/metrica_tesi', 'Felipehonorato/eer', 'Fritz02/execution_accuracy', 'GMFTBY/d

The `dataset` object itself is [`DatasetDict`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasetdict), which contains one key for the training, validation and test set (with more keys for the mismatched validation and test set in the special case of `mnli`).

In [13]:


def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [14]:
# show_random_elements(dataset["train"], num_examples=3)

You can call its `compute` method with your predictions and labels directly and it will return a dictionary with the metric(s) value:

In [15]:
import numpy as np

fake_preds = np.random.randint(0, 2, size=(64,))
fake_labels = np.random.randint(0, 2, size=(64,))
metric.compute(predictions=fake_preds, references=fake_labels)

{'mse': 0.40625}

## Preprocessing the data

Before we can feed those texts to our model, we need to preprocess them. This is done by a 🤗 Transformers `Tokenizer` which will (as the name indicates) tokenize the inputs (including converting the tokens to their corresponding IDs in the pretrained vocabulary) and put it in a format the model expects, as well as generate the other inputs that model requires.

To do all of this, we instantiate our tokenizer with the `AutoTokenizer.from_pretrained` method, which will ensure:

- we get a tokenizer that corresponds to the model architecture we want to use,
- we download the vocabulary used when pretraining this specific checkpoint.

That vocabulary will be cached, so it's not downloaded again the next time we run the cell.

In [16]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


We pass along `use_fast=True` to the call above to use one of the fast tokenizers (backed by Rust) from the 🤗 Tokenizers library. Those fast tokenizers are available for almost all models, but if you got an error with the previous call, remove that argument.

You can directly call this tokenizer on one sentence or a pair of sentences:

In [17]:
tokenizer("Hello, this one sentence!", "And this sentence goes with it.")

{'input_ids': [0, 31414, 6, 42, 65, 3645, 328, 2, 2, 2409, 42, 3645, 1411, 19, 24, 4, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

Depending on the model you selected, you will see different keys in the dictionary returned by the cell above. They don't matter much for what we're doing here (just know they are required by the model we will instantiate later), you can learn more about them in [this tutorial](https://huggingface.co/transformers/preprocessing.html) if you're interested.

To preprocess our dataset, we will thus need the names of the columns containing the sentence(s). The following dictionary keeps track of the correspondence task to column names:

In [18]:

task_to_keys = {
    'stb': ("text", None),
    'suicide_risk': ("text", None),
}

We can double check it does work on our current dataset:

In [19]:
sentence1_key, sentence2_key = task_to_keys[task]



if sentence2_key is None:
    print(f"Sentence: {dataset['train'][0][sentence1_key]}")
else:
    print(f"Sentence 1: {dataset['train'][0][sentence1_key]}")
    print(f"Sentence 2: {dataset['train'][0][sentence2_key]}")

Sentence: Hey, I'm about to off myself and want somebody to stop me. Like I said, I'm feeling terrible. I'm Worth. Yeah, razors. Yeah, in my pocket. Okay, that's okay. I put them in the couch. Right now. My life was perfect, and I fucked it up by being a drunk piece of shit. I was gonna get the medicine I need, hormones. But now I've risked all that. Okay, I go over to my friends house. We get drunk. I wake up, my mom nursing me in bed. I'm afraid what my parents will do to me. I want to see the doctor, if I can't get that, I'll kill myself. My parents, I'm afraid it's how they'll punish me. Some good, some bad. If my parents [scrubbed] me this medicine, I'll kill myself to get back at them. Yeah, a lot, especially when trying to get this medicine. I tried and failed to hang myself once. Stupid stupid queer. Thank you, that means a lot. Better, but still not good. My mom said she probably won't take my medicine away. If she does take it away, I will kill myself. Yeah, we've talked a bi

In [20]:
def preprocess_function(examples):
    if sentence2_key is None:
        return tokenizer(examples[sentence1_key], truncation=True)
    return tokenizer(examples[sentence1_key], examples[sentence2_key], truncation=True)

This function works with one or several examples. In the case of several examples, the tokenizer will return a list of lists for each key:

In [21]:
sentence1_key, sentence2_key = task_to_keys[task]

preprocess_function(dataset['train'][:5])

{'input_ids': [[0, 13368, 6, 38, 437, 59, 7, 160, 2185, 8, 236, 4909, 7, 912, 162, 4, 2011, 38, 26, 6, 38, 437, 2157, 6587, 4, 38, 437, 13399, 4, 8976, 6, 910, 1222, 994, 4, 8976, 6, 11, 127, 7524, 4, 8487, 6, 14, 18, 8578, 4, 38, 342, 106, 11, 5, 16433, 4, 5143, 122, 4, 1308, 301, 21, 1969, 6, 8, 38, 42647, 24, 62, 30, 145, 10, 10789, 2125, 9, 15328, 4, 38, 21, 6908, 120, 5, 6150, 38, 240, 6, 25641, 4, 125, 122, 38, 348, 25892, 70, 14, 4, 8487, 6, 38, 213, 81, 7, 127, 964, 790, 4, 166, 120, 10789, 4, 38, 3874, 62, 6, 127, 3795, 8701, 162, 11, 3267, 4, 38, 437, 6023, 99, 127, 1041, 40, 109, 7, 162, 4, 38, 236, 7, 192, 5, 3299, 6, 114, 38, 64, 75, 120, 14, 6, 38, 581, 3549, 2185, 4, 1308, 1041, 6, 38, 437, 6023, 24, 18, 141, 51, 581, 15392, 162, 4, 993, 205, 6, 103, 1099, 4, 318, 127, 1041, 646, 3866, 31413, 5134, 742, 162, 42, 6150, 6, 38, 581, 3549, 2185, 7, 120, 124, 23, 106, 4, 8976, 6, 10, 319, 6, 941, 77, 667, 7, 120, 42, 6150, 4, 38, 1381, 8, 1447, 7, 6713, 2185, 683, 4, 44462, 1

To apply this function on all the sentences (or pairs of sentences) in our dataset, we just use the `map` method of our `dataset` object we created earlier. This will apply the function on all the elements of all the splits in `dataset`, so our training, validation and testing data will be preprocessed in one single command.

In [22]:
encoded_dataset = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/3280 [00:00<?, ? examples/s]

Map:   0%|          | 0/823 [00:00<?, ? examples/s]

Map:   0%|          | 0/1071 [00:00<?, ? examples/s]

Even better, the results are automatically cached by the 🤗 Datasets library to avoid spending time on this step the next time you run your notebook. The 🤗 Datasets library is normally smart enough to detect when the function you pass to map has changed (and thus requires to not use the cache data). For instance, it will properly detect if you change the task in the first cell and rerun the notebook. 🤗 Datasets warns you when it uses cached files, you can pass `load_from_cache_file=False` in the call to `map` to not use the cached files and force the preprocessing to be applied again.

Note that we passed `batched=True` to encode the texts by batches together. This is to leverage the full benefit of the fast tokenizer we loaded earlier, which will use multi-threading to treat the texts in a batch concurrently.

## Fine-tuning the model with or without hyperparameter search

In [23]:
from sklearn import metrics
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, roc_auc_score, recall_score, precision_score, f1_score, auc, precision_recall_curve, classification_report
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import mean_squared_error





# def compute_metrics(eval_pred):
#     from evaluate import load
#     predictions, labels = eval_pred
#     metric = load('mse')
#     return metric.compute(predictions=predictions, references=labels, squared=False)


# def compute_metrics(eval_pred):
#     predictions, labels = eval_pred
#     if task != "stsb":
#         predictions = np.argmax(predictions, axis=1)
#     else:
#         predictions = predictions[:, 0]
#     return metric.compute(predictions=predictions, references=labels)

def metrics_report_cm(y_true, y_pred, output_dir, model_name, ts, save=True):

    cm = confusion_matrix(y_true, y_pred,normalize=None)
    cm_df = pd.DataFrame(cm, index=['SITB-', 'SITB+'], columns=['SITB-', 'SITB+'])
    cm_df_meaning = pd.DataFrame([['TN', 'FP'],['FN','TP']], index=['SITB-', 'SITB+'], columns=['SITB-', 'SITB+'])

    cm_norm = confusion_matrix(y_true, y_pred,normalize='all')
    cm_norm = (cm_norm*100).round(2)
    cm_df_norm = pd.DataFrame(cm_norm, index=['SITB-', 'SITB+'], columns=['SITB-', 'SITB+'])


    plt.rcParams['figure.figsize'] = [4,4]
    cm_display = ConfusionMatrixDisplay(cm_norm,display_labels=['SITB-', 'SITB+']).plot()
    # todo save

    if save:
        cm_df_meaning.to_csv(output_dir+f'cm_meaning_{model_name}_{ts}.csv')
        cm_df.to_csv(output_dir+f'cm_{model_name}_{ts}.csv')
        cm_df_norm.to_csv(output_dir+f'cm_norm_{model_name}_{ts}.csv')



    return cm_df_meaning, cm_df, cm_df_norm






def metrics_report_classification_report(y_true, y_pred,y_pred_proba_1, output_dir, model_name, ts):

    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    np.set_printoptions(suppress=True)
    roc_auc = roc_auc_score(y_true,y_pred)
    f1 = f1_score(y_true,y_pred)

    # calculate precision and recall for each threshold
    lr_precision, lr_recall, thresholds = precision_recall_curve(y_true, y_pred_proba_1)

    # TODO: add best threshold
    fscore = (2 * lr_precision * lr_recall) / (lr_precision + lr_recall)
    fscore[np.isnan(fscore)] = 0
    ix = np.argmax(fscore)
    best_threshold = thresholds[ix].item()


    pr_auc = auc(lr_recall, lr_precision)
    # AU P-R curve is also approximated by avg. precision
    # avg_pr = metrics.average_precision_score(y_true,y_pred_proba_1)

    sensitivity = recall_score(y_true,y_pred)
    specificity = tn / (tn+fp) # OR: recall_score(y_true,y_pred, pos_label=0)
    precision = precision_score(y_true,y_pred)

    results = pd.DataFrame([sensitivity, specificity,precision,f1, roc_auc,pr_auc, best_threshold],
                        index = ['Sensitivity', 'Specificity', 'Precision', 'F1', 'ROC AUC','PR AUC', 'Best th PR AUC']).T.round(2)

    results.to_csv(output_dir+f'results_{model_name}_{ts}.csv')
    return results



from matplotlib import pyplot as plt
from sklearn import metrics
from scipy.stats import pearsonr, spearmanr


def regression_report(y_test,y_pred,y_train=None,gridsearch=None, best_params=None,feature_vector=None,model_name=None,metrics_to_report = 'all', plot = True, save_fig_path = None, round_to = 2):
    '''
    metrics = {'all', ['MAE','RMSE','rho', 'Best parameters']
    }
    '''

    # Metrics
    # https://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics
    rmse = metrics.mean_squared_error(y_test, y_pred, squared=False )
    mae = metrics.mean_absolute_error(y_test, y_pred)
    r2 = metrics.r2_score(y_test, y_pred)
    r, p = pearsonr(y_test, y_pred)
    rho, p = spearmanr(y_test, y_pred)


    results_dict = {
        'Model':f"{feature_vector} {model_name}",
        'y_train_min': np.min(y_train),
        'y_train_max': np.max(y_train),
        'RMSE':np.round(rmse,round_to ),
        'MAE':np.round(mae,round_to ),
        'R^2':np.round(r2,round_to ),
        'r':np.round(r,round_to ),
        'rho':np.round(r,round_to ),
        'gridsearch':gridsearch,
        'Best parameters': str(best_params),
        }
    results = pd.DataFrame(results_dict, index=[model_name]).round(3)
    # results_all.append(results)
    if metrics != 'all':
        if 'RMSE per value' in metrics_to_report or 'MAE per value' in metrics_to_report:
            y_pred_test = {}
            y_pred_test['RMSE per value'] = []
            y_pred_test['MAE per value'] = []
            for value in np.unique(y_test):
                y_pred_test_i = [[pred,test] for pred,test in zip(y_pred,y_test) if test == value]
                y_pred_test[value] = np.array(y_pred_test_i)
                y_pred_i = [n[0] for n in y_pred_test_i]
                y_test_i = [n[1] for n in y_pred_test_i]
                rmse_i = metrics.mean_squared_error(y_test_i, y_pred_i, squared=False )
                mae_i = metrics.mean_absolute_error(y_test_i, y_pred_i)
                y_pred_test['RMSE per value'].append(np.round(rmse_i,round_to ))
                y_pred_test['MAE per value'].append(np.round(mae_i,round_to ))
            print(y_pred_test['RMSE per value'])
            results_dict.update({
            'RMSE per value':f"{results_dict['RMSE']} {y_pred_test['RMSE per value']}",
            'MAE per value':f"{results_dict['MAE']} {y_pred_test['MAE per value']}"
            })
            metrics_to_report_2 = metrics_to_report.copy()
            metrics_to_report_2.remove('RMSE') #redudant
            metrics_to_report_2.remove('MAE') #redudant
            results = pd.DataFrame(results_dict, index=[model_name]) # replace with updated metrics
            results = results[metrics_to_report_2]
        else:
            results = pd.DataFrame(results_dict, index=[model_name]) # replace with updated metrics
            results = results[metrics_to_report]






    # Plot result for a regression task: true value vs predicted values
    # ============================================================
    plt.clf()
    plt.style.use('default')  # Example of applying the 'ggplot' style
    plt.scatter(y_test, y_pred, alpha = 0.1)
    plt.title(f"RMSE: {results_dict['RMSE']}\nMAE: {results_dict['MAE']}\nrho: {results_dict['rho']}"
            )
    plt.xlabel('True values')
    plt.ylabel('Predicted values')

    plt.tight_layout()
    if save_fig_path:
        plt.savefig(save_fig_path+'.png', dpi=150)
    plt.show()
    return results





Now that our data is ready, we can download the pretrained model and fine-tune it. Since all our tasks are about sentence classification, we use the `AutoModelForSequenceClassification` class. Like with the tokenizer, the `from_pretrained` method will download and cache the model for us. The only thing we have to specify is the number of labels for our problem (which is always 2, except for STS-B which is a regression problem and MNLI where we have 3 labels):

### Custom Trainer to add class weights if you have class imbalance

In [24]:

from torch import nn
from transformers import Trainer

from sklearn.utils.class_weight import compute_class_weight

class_weights =  compute_class_weight(class_weight = "balanced", classes= np.unique(y_train), y= y_train)
print(class_weights)

class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        # forward pass
        outputs = model(**inputs)
        logits = outputs.get("logits")
        # compute custom loss (suppose one has 3 labels with different weights)
        loss_fct = nn.CrossEntropyLoss(weight=torch.tensor(class_weights, device=model.device).float())
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss

[0.98944193 1.03339635 0.97881229]


### define or download model and define training arguments

In [25]:
%%time

# Finetuning no hyperparameter search:
# distilroberta 18m (4 epochs)
# roberta-base 35m (3 epochs)


def compute_metrics(eval_pred):

    predictions, labels = eval_pred
    if regression:
      rmse = mean_squared_error(labels, predictions, squared=False)
      return {"rmse": rmse}
    else:
      if task != "stb":
          predictions = np.argmax(predictions, axis=1)
      else:
          predictions = predictions[:, 0]
      return metric.compute(predictions=predictions, references=labels)

model_name = model_checkpoint.split("/")[-1]

args = TrainingArguments(
    f"{model_name}-finetuned-{task}",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=epochs,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,

  push_to_hub=False,

)


# specify model and model_init

# Regression also works with AutoModelForSequenceClassification but with num_labels =1 . Then the compute_metrics will be different
if do_hyperparameter_search:
    model = None
    def create_model_init():
        return AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

    model_init = create_model_init


else:
    model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)
    model_init=None


def define_trainer(encoded_train_set, encoded_val_set, model=None, model_init=None, compute_metrics = None, imbalanced = imbalanced):
    if not imbalanced:
        trainer = Trainer(
            model = model,
            model_init = model_init,
            args = args,
            train_dataset=encoded_train_set,
            eval_dataset=encoded_val_set,
            tokenizer=tokenizer,
          compute_metrics=compute_metrics,
        )

    else:
        # here we use a CustomTrainer for class weights
        trainer = CustomTrainer(
            model = model,
            model_init = model_init,
            args = args,
            train_dataset=encoded_train_set,
            eval_dataset=encoded_val_set,
            tokenizer=tokenizer,
            compute_metrics=compute_metrics
        )
    return trainer



CPU times: user 4.11 ms, sys: 26 µs, total: 4.14 ms
Wall time: 11.9 ms


The warning is telling us we are throwing away some weights (the `vocab_transform` and `vocab_layer_norm` layers) and randomly initializing some other (the `pre_classifier` and `classifier` layers). This is absolutely normal in this case, because we are removing the head used to pretrain the model on a masked language modeling objective and replacing it with a new head for which we don't have pretrained weights, so the library warns us we should fine-tune this model before using it for inference, which is exactly what we are going to do.

To instantiate a `Trainer`, we will need to define two more things. The most important is the [`TrainingArguments`](https://huggingface.co/transformers/main_classes/trainer.html#transformers.TrainingArguments), which is a class that contains all the attributes to customize the training. It requires one folder name, which will be used to save the checkpoints of the model, and all other arguments are optional:

Here we set the evaluation to be done at the end of each epoch, tweak the learning rate, use the `batch_size` defined at the top of the notebook and customize the number of epochs for training, as well as the weight decay. Since the best model might not be the one at the end of training, we ask the `Trainer` to load the best model it saved (according to `metric_name`) at the end of training.

The last argument to setup everything so we can push the model to the [Hub](https://huggingface.co/models) regularly during training. Remove it if you didn't follow the installation steps at the top of the notebook. If you want to save your model locally in a name that is different than the name of the repository it will be pushed, or if you want to push your model under an organization and not your name space, use the `hub_model_id` argument to set the repo name (it needs to be the full name, including your namespace: for instance `"sgugger/bert-finetuned-mrpc"` or `"huggingface/bert-finetuned-mrpc"`).

The last thing to define for our `Trainer` is how to compute the metrics from the predictions. We need to define a function for this, which will just use the `metric` we loaded earlier, the only preprocessing we have to do is to take the argmax of our predicted logits (our just squeeze the last axis in the case of STS-B):

You might wonder why we pass along the `tokenizer` when we already preprocessed our data. This is because we will use it once last time to make all the samples we gather the same length by applying padding, which requires knowing the model's preferences regarding padding (to the left or right? with which token?). The `tokenizer` has a pad method that will do all of this right for us, and the `Trainer` will use it. You can customize this part by defining and passing your own `data_collator` which will receive the samples like the dictionaries seen above and will need to return a dictionary of tensors.

We can now finetune our model by just calling the `train` method:

# Train and save model

In [ ]:
%%time

encoded_train_set = encoded_dataset["train"]
encoded_val_set = encoded_dataset[validation_key]

if do_hyperparameter_search:
    if hyperparameter_shards:
        encoded_train_dataset_shards = encoded_train_set.shard(index=1, num_shards=hyperparameter_shards)
        trainer = define_trainer(encoded_train_dataset_shards, encoded_val_set, model=model, model_init=model_init , compute_metrics = compute_metrics,imbalanced=imbalanced)
    else:
        trainer = define_trainer(encoded_train_set, encoded_val_set, model=model, model_init=model_init , compute_metrics = compute_metrics,imbalanced=imbalanced)
    best_run = trainer.hyperparameter_search(n_trials=hyperparameter_search_n_trials,
                                             direction=hyperparameter_search_direction,
                                             backend='optuna',
                                             hp_space = optuna_hp_space, # or will use default hp_space for whatever the backend is
                                             )
    print('==== best run:', best_run)
    print()

    # re define the trainer with the entire dataset
    trainer = define_trainer(encoded_train_set, encoded_val_set, model=model, model_init=model_init , compute_metrics = compute_metrics, imbalanced=imbalanced)
    for n, v in best_run.hyperparameters.items():
        setattr(trainer.args, n, v)

# train on entire training set for the last time
trainer.train()

ts_i = datetime.datetime.utcnow().strftime('%y-%m-%dT%H-%M-%S')

if toy:
  output_dir_i = output_dir + f'results_{ts_i}_toy/'
else:
  output_dir_i = output_dir + f'results_{ts_i}/'
os.makedirs(output_dir_i, exist_ok=True)

trainer.save_model(output_dir_i)
# trainer.push_to_hub() # need to have added token above in TrainingArguments()

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.bias']
You should pr

In [ ]:
# # Load
# from transformers import AutoModel, AutoConfig
# config = AutoModelForSequenceClassification.from_pretrained(output_dir_i)
# model = AutoModelForSequenceClassification.from_pretrained(output_dir_i, config=config)

In [ ]:
# model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)
# model_init=None


trainer = define_trainer(encoded_train_set, encoded_val_set, model=model, model_init=model_init, compute_metrics = compute_metrics, imbalanced = imbalanced)

In [ ]:
# best_params = f"weight_decay: {trainer.args.weight_decay}; num_train_epochs: {trainer.args.num_train_epochs}; learning_rate: {trainer.args.learning_rate}"
# best_params = "weight_decay: 6.8120818721882775e-06; num_train_epochs: 4; learning_rate: 1.094605203258915e-06"
# best_params


In [ ]:

best_params = str(list(best_run.hyperparameters.items()))
best_params

The best run was:



# Evaluate on test set

In [ ]:
import datetime


results = []




# Regression
if regression:
  feature_vector = None
  gridsearch = True

  y_pred = trainer.predict(test_dataset=encoded_dataset["test"]).predictions
  y_pred = y_pred.reshape(1,-1)[0]


  y_pred_df = pd.DataFrame(y_pred)
  y_pred_df.to_csv(output_dir_i+f'y_pred_{model_name}_gridsearch-{gridsearch}_{ts_i}.csv', index=False)
  path = output_dir_i + f'scatter_{model_name}_gridsearch-{gridsearch}_{ts_i}'

  # Performance
  results_i =regression_report(y_test,y_pred,y_train=y_train,
                              metrics_to_report = metrics_to_report,
                                gridsearch=gridsearch,
                              best_params=best_params,feature_vector=feature_vector,model_name=model_name, plot = True, save_fig_path = path, round_to = 2)
  results_i.to_csv(output_dir_i + f'results_{model_name}_gridsearch-{gridsearch}_{ts_i}.csv')
  display(results_i)
  results.append(results_i)
  results_df = pd.concat(results)
  results_df = results_df.reset_index(drop=True)
  results_df.to_csv(output_dir_i + f'results_{ts_i}.csv', index=False)

else:
  # Classification
  y_pred_proba = trainer.predict(test_dataset=encoded_dataset["test"]).predictions

  y_pred_proba = np.array(y_pred_proba)
  y_pred_proba_1 = y_pred_proba[:,1]
  y_pred = [np.argmax(n) for n in y_pred_proba]

  test['y_pred_proba_0'] = y_pred_proba[:,0]
  test['y_pred_proba_1'] = y_pred_proba[:,1]
  test['y_pred'] = y_pred


  test.to_csv(output_dir_i+'test_predictions.csv', index=False)

  # y_pred = [int(np.argmax(n)) for n in y_pred_proba]
  print(metrics.classification_report(y_pred=y_pred, y_true=y_test))
  results = {}

  clf_report_sklearn = metrics.classification_report(y_test,y_pred, output_dict=False) #evaluate #different than the output of cross_validate() above.
  cm_df_meaning, cm_df, cm_df_norm = metrics_report_cm(y_test, y_pred, output_dir_results, model_name, ts, save=True)
  clf_report = metrics_report_classification_report(y_test,y_pred,y_pred_proba_1, output_dir_results, model_name, ts)
  # scores = cross_validate(pipe, X, y, scoring=['f1','precision', 'recall'], cv=cv, return_train_score=False) #train and evaluate
  results[model_name] = {
      'clf_report': clf_report,
      'cm_df_meaning': cm_df_meaning,
      'cm_df': cm_df,
      'cm_df_norm': cm_df_norm,
      'clf_report_sklearn': clf_report_sklearn

  }
  for k, v in results.get(model_name).items():
    print(k)
    if k != 'clf_report_sklearn':
      display(v)
    else:
      print(v)
    print()


In [ ]:
# trainer.evaluate(eval_dataset=encoded_dataset["test"])
# trainer.evaluate() # this is done on the validation set specificed in the Trainer()

In [ ]:
# load model:
#  - https://discuss.huggingface.co/t/how-to-save-my-model-to-use-it-later/20568/6
# - model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)
# - then just load the trainer, but instead of training you just predict (here I'm training to fine tune, but if we load the fine tuned model , there's no need)


